In [2052]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2053]:
base = pd.read_csv('iec6100429.csv')

In [2054]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe

In [2055]:
x.head()

,Amostra 1,Amostra 2,Amostra 3,Amostra 4,Amostra 5,Amostra 6,Amostra 7,Amostra 8,Amostra 9,Amostra 10,...,Amostra 182,Amostra 183,Amostra 184,Amostra 185,Amostra 186,Amostra 187,Amostra 188,Amostra 189,Amostra 190,Amostra 191
0,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
1,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
2,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
4,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3


In [2056]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape

(92,)

In [2057]:
y = y.values.reshape(-1,1)
y.shape

(92, 1)

In [2058]:
y = onehot.fit_transform(y).toarray()

In [2059]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.5)

In [2060]:
x.shape, x_teste.shape, y.shape, y_teste.shape

((92, 191), (46, 191), (92, 7), (46, 7))

In [2061]:
neuronios_entrada = x.shape[1]
neuronios_entrada

191

In [2062]:
neuronios_oculta = 350
neuronios_oculta

350

In [2063]:
neuronios_saida = y.shape[1]
neuronios_saida

7

In [2064]:
w = {'oculta': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta, neuronios_saida]))}
w

{'oculta': <tf.Variable 'Variable_372:0' shape=(191, 350) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_373:0' shape=(350, 7) dtype=float32_ref>}

In [2065]:
b = {'oculta': tf.Variable(tf.random_normal([neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}
b

{'oculta': <tf.Variable 'Variable_374:0' shape=(350,) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_375:0' shape=(7,) dtype=float32_ref>}

In [2066]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [2067]:
def mlp(x, w, bias):
    camada_oculta = tf.add(tf.matmul(x, w['oculta']), b['oculta'])
    camada_oculta_ativacao = tf.nn.relu(camada_oculta)
    camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, w['saida']), b['saida'])
    return camada_saida

In [2068]:
modelo = mlp(xph, w, b)

In [2069]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(erro)

In [2070]:
batch_size = 8
batch_total = int(len(x_treinamento) / batch_size)
batch_total, (len(x_treinamento))

(5, 46)

In [2071]:
x_batches = np.array_split(x_treinamento, batch_total)

In [2072]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(10000):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 500 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])
    
    previsoes = tf.nn.softmax(mlp(xph, w_final, b_final))
    previsoes_corretas = tf.equal(tf.argmax(previsoes, 1), tf.argmax(yph, 1))
    taxa_acerto = tf.reduce_mean(tf.cast(previsoes_corretas, tf.float32))
    print(sess.run(taxa_acerto, feed_dict = {xph: x_teste, yph: y_teste}))

Época: 1 erro: 36317.94405517579
Época: 501 erro: 0.3193332600705617
Época: 1001 erro: 0.0014293854561401532
Época: 1501 erro: 0.0008504051169438754
Época: 2001 erro: 0.34796466454863545
Época: 2501 erro: 0.007227533945115283
Época: 3001 erro: 0.7494567902433857
Época: 3501 erro: 1.8690584262212107
Época: 4001 erro: 8.162536687450483e-05
Época: 4501 erro: 6.530479149660096e-05
Época: 5001 erro: 3.948237491613327e-05
Época: 5501 erro: 1.799748807798096e-05
Época: 6001 erro: 1.36292445488898
Época: 6501 erro: 0.652160006099257
Época: 7001 erro: 1.3605612821265824e-06
Época: 7501 erro: 1.0866501497730497e-06
Época: 8001 erro: 1.0532720125411287e-06
Época: 8501 erro: 1.0217481985819176e-06
Época: 9001 erro: 9.213490670845203e-07
Época: 9501 erro: 7.062451913952828e-07
0.8043478


In [2073]:
previsoes_teste = mlp(xph, w_final, b_final)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(previsoes_teste, feed_dict = {xph: x_teste})
    r2 = sess.run(tf.nn.softmax(r1))
    r3 = sess.run(tf.argmax(r2, 1))
    
y_teste2 = np.argmax(y_teste, 1)
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(y_teste2, r3)
taxa_acerto

0.7608695652173914